In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.appName("myApp").getOrCreate()

In [15]:
#Reading txt and csv files
userLibRDD = sparkSession.sparkContext.textFile("./mod_users_libraries.txt")
stopWordRDD = sparkSession.sparkContext.textFile("./stopwords_en.txt")
papersRDD = sparkSession.sparkContext.textFile("./papers.csv")

In [16]:
#Exercise 1.2
#Creating a pair RDD from user_libraries.txt using the user hash as key and liked paper(s) as value(s)
userRatingsRDD = userLibRDD.map(lambda line: (line.split(';')[0], line.split(';')[1]))
#Displaying first 'n' elements of dataset
userRatingsRDD.take(3)

[('28d3f81251d94b09735497477a5e4e02',
  '3929762,503574,5819422,4238883,5788061,462949,635215,635216,4810441,3481823,4165233,3366480,5984302,4238942,5490453,4636156,5996865,4194836,5828780,4450195'),
 ('d0c9aaa788153daeaf1f1538b3d46bbb',
  '2080631,6343346,5184704,7756088,2653863,6607628,4236212,1277953,226864,3140015,8806369,311570,5687747,767516,4781370,2841637,2445106,1959511,2688186,2363430,6614346,853030,5336762,4226226,239571,4089758,4140337,913868,7562861,3190274,2782576,12571584,2049617,5761055,5441098,3466838,2080691,1805577,7570111,5760287,2855355,3281547,1012525,3512183,678653'),
 ('f05bcffe7951de9e5a32fff4a42eb088',
  '1158654,478707,12054725,6670515,781057,13329754,781055,2825,553840,988447,12938139,12037219,12788583,6595566,4027225,920055,3129258,2242776,3112352,144287,706033,525396,622633,9172127,7357993,230211,12790816,503161,12937120,942241,166220,8493138,2945819,227173,11191048,949352,227096,11597926,921623,833638,882809,12738996,99,7154210,11852474,72879,10723139,119

In [17]:
#Exercise 1.2
#converting the user ratings into key value pairs rdd
userRatingsList = []
for keyListVal in userRatingsRDD.collect():
    for val in keyListVal[1].split(','):
        userRatingsList.append((keyListVal[0],val))

#print(userRatingsList[0:30])  
userRatingsKVPairRDD = sparkSession.sparkContext.parallelize(userRatingsList)
userRatingsKVPairRDD.take(5)

[('28d3f81251d94b09735497477a5e4e02', '3929762'),
 ('28d3f81251d94b09735497477a5e4e02', '503574'),
 ('28d3f81251d94b09735497477a5e4e02', '5819422'),
 ('28d3f81251d94b09735497477a5e4e02', '4238883'),
 ('28d3f81251d94b09735497477a5e4e02', '5788061')]

In [18]:
#Exercise 1.2
#Creating a pair RDD from papers.csv using paper_id as key and abstract as value(s)
paperTermsRDD = papersRDD.map(lambda line: (line.split(',')[0], ",".join(line.split(',')[14:])))
#Displaying first 'n' elements of dataset
#saveRDDAsFile(paperTermsRDD)
paperTermsRDD.take(3)

[('80546',
  '"the genetic code has been regarded as arbitrary in the sense that the codon-amino acid assignments could be different than they actually are. this general idea has been spelled out differently by previous, often rather implicit accounts of arbitrariness. they have drawn on the frozen accident theory, on evolutionary contingency, on alternative causal pathways, and on the absence of direct stereochemical interactions between codons and amino acids. it has also been suggested that the arbitrariness of the genetic code justifies attributing semantic information to macromolecules, notably to {dna}. i argue that these accounts of arbitrariness are unsatisfactory. i propose that the code is arbitrary in the sense of jacques monod\'s concept of chemical arbitrariness: the genetic code is arbitrary in that any codon requires certain chemical and structural properties to specify a particular amino acid, but these properties are not required in virtue of a principle of chemistry. 

In [306]:
#Exercise 1.3
#Creating a broadcast variable holding stop words
stopWordBrdcast = sparkSession.sparkContext.broadcast(stopWordRDD.collect())
#stopWordBrdcast.value


In [19]:
#Exercise 1.3
#Create RDD containg the paperID as key and UserHash as value
paperIDUserHashRDD = userRatingsKVPairRDD.map(lambda x: (x[1], x[0]))
paperIDUserHashRDD.take(5)


[('3929762', '28d3f81251d94b09735497477a5e4e02'),
 ('503574', '28d3f81251d94b09735497477a5e4e02'),
 ('5819422', '28d3f81251d94b09735497477a5e4e02'),
 ('4238883', '28d3f81251d94b09735497477a5e4e02'),
 ('5788061', '28d3f81251d94b09735497477a5e4e02')]

In [20]:
#Using Join to match paperIDUserHashRDD with paperTermsRDD
joinResultRDD = paperIDUserHashRDD.join(paperTermsRDD)
joinResultRDD.take(5)

[('503574', ('28d3f81251d94b09735497477a5e4e02', '')),
 ('5788061',
  ('28d3f81251d94b09735497477a5e4e02',
   ' and its impact on {cmb} anomalies",')),
 ('4238942', ('28d3f81251d94b09735497477a5e4e02', '')),
 ('5490453', ('28d3f81251d94b09735497477a5e4e02', '')),
 ('5828780', ('28d3f81251d94b09735497477a5e4e02', ''))]

In [21]:
#function 
def joinListElements(x):
    return ''.join(x)
#Create new RDD containing userHash and abstract from RDD of join result
#Using groupByKey() and mapValues to group values for each key into a single list
transJoinResultRDD = joinResultRDD.map(lambda x: (x[1][0], x[1][1].strip())).groupByKey().mapValues(list).mapValues(joinListElements)
transJoinResultRDD.take(10)

[('f05bcffe7951de9e5a32fff4a42eb088',
  'a protein domain database for functional characterization and annotation.","{prosite} consists of documentation entries describing protein domains, families and functional sites, as well as associated patterns and profiles to identify them. it is complemented by {prorule}, a collection of rules based on profiles and patterns, which increases the discriminatory power of these profiles and patterns by providing additional information about functionally and/or structurally critical amino acids. {prosite} is largely used for the annotation of domain features of {uniprotkb}/{swiss-prot} entries. among the 983 ({dna}-binding) domains, repeats and zinc fingers present in {swiss-prot} (release 57.8 of 22 september 2009), 696 ( approximately 70\\%) are annotated with {prosite} descriptors using information from {prorule}. in order to allow better functional characterization of domains, {prosite} developments focus on subfamily specific profiles and a new

In [22]:
#splitting the values every time a blank space is encountered
splitRDD = transJoinResultRDD.map(lambda x: (x[0], x[1].split(' ')))
splitRDD.take(2)

[('f05bcffe7951de9e5a32fff4a42eb088',
  ['a',
   'protein',
   'domain',
   'database',
   'for',
   'functional',
   'characterization',
   'and',
   'annotation.","{prosite}',
   'consists',
   'of',
   'documentation',
   'entries',
   'describing',
   'protein',
   'domains,',
   'families',
   'and',
   'functional',
   'sites,',
   'as',
   'well',
   'as',
   'associated',
   'patterns',
   'and',
   'profiles',
   'to',
   'identify',
   'them.',
   'it',
   'is',
   'complemented',
   'by',
   '{prorule},',
   'a',
   'collection',
   'of',
   'rules',
   'based',
   'on',
   'profiles',
   'and',
   'patterns,',
   'which',
   'increases',
   'the',
   'discriminatory',
   'power',
   'of',
   'these',
   'profiles',
   'and',
   'patterns',
   'by',
   'providing',
   'additional',
   'information',
   'about',
   'functionally',
   'and/or',
   'structurally',
   'critical',
   'amino',
   'acids.',
   '{prosite}',
   'is',
   'largely',
   'used',
   'for',
   'the',
   'a

In [23]:
#counting the frequency of each individual words used by every user
splitRDD2 = splitRDD.flatMapValues(lambda x: x).map(lambda x: (x,1)).reduceByKey(lambda a, b: a + b)
#splitRDD2.persist()
splitRDD2.take(10)

[(('f05bcffe7951de9e5a32fff4a42eb088', 'database'), 61),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'functional'), 40),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'of'), 1050),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'documentation'), 4),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'families'), 5),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'as'), 181),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'identify'), 21),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'rules'), 3),
 (('f05bcffe7951de9e5a32fff4a42eb088', 'power'), 8),
 (('f05bcffe7951de9e5a32fff4a42eb088', '{uniprotkb}/{swiss-prot}'), 1)]

In [45]:
rearrangedRDD = splitRDD2.map(lambda x: ((x[0][0], (x[0][1], x[1]))))
rearrangedRDD.take(10)

[('f05bcffe7951de9e5a32fff4a42eb088', ('database', 61)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('functional', 40)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('of', 1050)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('documentation', 4)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('families', 5)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('as', 181)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('identify', 21)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('rules', 3)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('power', 8)),
 ('f05bcffe7951de9e5a32fff4a42eb088', ('{uniprotkb}/{swiss-prot}', 1))]

In [56]:
groupedSortedRDD = rearrangedRDD.groupByKey().map(lambda x: (x[0], sorted(list(x[1]), key=lambda x: x[1], reverse=True)))
groupedSortedRDD.take(5)

[('f05bcffe7951de9e5a32fff4a42eb088',
  [('the', 1157),
   ('of', 1050),
   ('and', 862),
   ('a', 541),
   ('to', 502),
   ('in', 418),
   ('for', 384),
   ('is', 317),
   ('data', 207),
   ('that', 206),
   ('as', 181),
   ('are', 174),
   ('we', 157),
   ('with', 153),
   ('from', 139),
   ('by', 117),
   ('on', 111),
   ('be', 108),
   ('an', 107),
   ('this', 107),
   ('can', 105),
   ('information', 97),
   ('gene', 90),
   ('binding', 86),
   ('analysis', 85),
   ('biological', 84),
   ('at', 79),
   ('have', 78),
   ('available', 77),
   ('protein', 77),
   ('sequence', 72),
   ('or', 71),
   ('which', 71),
   ('transcription', 65),
   ('these', 65),
   ('their', 62),
   ('database', 61),
   ('genes', 58),
   ('also', 56),
   ('sites', 56),
   ('such', 56),
   ('has', 55),
   ('regulatory', 51),
   ('all', 51),
   ('been', 50),
   ('it', 50),
   ('new', 50),
   ('motif', 50),
   ('used', 48),
   ('sequences', 47),
   ('networks', 46),
   ('different', 45),
   ('provides', 44),


In [307]:
topTenGroupRDD = groupedSortedRDD.map(lambda x: ([x[0]] ,x[1][0:10]))
topTenGroupRDD.take(5)

[(['f05bcffe7951de9e5a32fff4a42eb088'],
  [('the', 1157),
   ('of', 1050),
   ('and', 862),
   ('a', 541),
   ('to', 502),
   ('in', 418),
   ('for', 384),
   ('is', 317),
   ('data', 207),
   ('that', 206)]),
 (['beaca02b21b7cad6cb738c5e2682af8d'],
  [('the', 2810),
   ('of', 2747),
   ('and', 1653),
   ('in', 1278),
   ('to', 1174),
   ('a', 1164),
   ('that', 675),
   ('for', 649),
   ('is', 601),
   ('we', 568)]),
 (['dacb9d5d7b6e1b8090aa2d4cf6542ea1'],
  [('the', 1443),
   ('of', 1346),
   ('and', 1133),
   ('to', 733),
   ('in', 684),
   ('a', 549),
   ('for', 367),
   ('that', 256),
   ('is', 252),
   ('on', 192)]),
 (['6dd817ab0c2f730fc206faf5f5847bbb'],
  [('the', 3244),
   ('of', 2407),
   ('and', 1966),
   ('to', 1570),
   ('a', 1443),
   ('in', 1289),
   ('for', 804),
   ('search', 788),
   ('that', 752),
   ('we', 700)]),
 (['37769fdc48d9a70ccf3a3245f7375230'],
  [('the', 353),
   ('of', 326),
   ('and', 265),
   ('to', 217),
   ('a', 185),
   ('in', 180),
   ('we', 134),


In [308]:
finalResultRDD = finalResultRDD.mapValues(lambda x: [t[0] for t in x])
finalResultRDD.take(10)

[(['f05bcffe7951de9e5a32fff4a42eb088'],
  ['the', 'of', 'and', 'a', 'to', 'in', 'for', 'is', 'data', 'that']),
 (['beaca02b21b7cad6cb738c5e2682af8d'],
  ['the', 'of', 'and', 'in', 'to', 'a', 'that', 'for', 'is', 'we']),
 (['dacb9d5d7b6e1b8090aa2d4cf6542ea1'],
  ['the', 'of', 'and', 'to', 'in', 'a', 'for', 'that', 'is', 'on']),
 (['6dd817ab0c2f730fc206faf5f5847bbb'],
  ['the', 'of', 'and', 'to', 'a', 'in', 'for', 'search', 'that', 'we']),
 (['37769fdc48d9a70ccf3a3245f7375230'],
  ['the', 'of', 'and', 'to', 'a', 'in', 'we', 'for', 'that', 'on']),
 (['589b870a611c25fa99bd3d7295ac0622'],
  ['the', 'and', 'to', 'of', 'in', 'a', 'wormhole', 'is', 'are', 'routing']),
 (['90f1a3e6fcdbf9bc550e866116bbcea5'],
  ['of', 'the', 'and', 'to', 'a', 'in', 'that', 'for', 'is', 'have']),
 (['6527bbd2024cf027a9ba75b3e28ed4db'],
  ['the', 'of', 'in', 'and', 'to', 'this', 'community', 'is', 'a', 'for']),
 (['d705113f772771b23bcd17303ee0860a'],
  ['the', 'of', 'and', '', 'in', 'a', 'to', 'as', 'for', 'that']

In [309]:
finalResultRDD.saveAsTextFile("output_files")